In [ ]:
import os
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
import scipy.io.wavfile as wavfile
import scipy
import scipy.fftpack as fftpk
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import sklearn
import re
# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/Chords/samples.zip' 

Archive:  /content/drive/MyDrive/Chords/samples.zip
replace samples/am0.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: samples/am0.wav         
  inflating: samples/am1.wav         
  inflating: samples/am2.wav         
  inflating: samples/am3.wav         
  inflating: samples/am4.wav         
  inflating: samples/c0.wav          
  inflating: samples/c1.wav          
  inflating: samples/c2.wav          
  inflating: samples/c3.wav          
  inflating: samples/c4.wav          
  inflating: samples/dm0.wav         
  inflating: samples/dm1.wav         
  inflating: samples/dm2.wav         
  inflating: samples/dm3.wav         
  inflating: samples/dm4.wav         
  inflating: samples/e0.wav          
  inflating: samples/e1.wav          
  inflating: samples/e2.wav          
  inflating: samples/e3.wav          
  inflating: samples/e4.wav          
  inflating: samples/g0.wav          
  inflating: samples/g1.wav          
  inflating: samples/g2.wav          
  inflati

In [ ]:
!pip install librosa

In [ ]:
import librosa
y, sr = librosa.load('/content/samples/am2.wav')
C = librosa.feature.mfcc(y=y, sr=sr)
C[0].mean()

-348.22144

In [ ]:
C.shape

(20, 162)

In [ ]:
np.average(C, axis = 1)

array([-348.22144  ,  160.14104  ,   55.223995 ,   28.682045 ,
         19.212156 ,   17.213095 ,   12.231215 ,    7.459236 ,
          0.6880066,   -8.070168 ,  -10.838573 ,   -9.788377 ,
         -7.789629 ,   -3.9034462,   -2.9333587,   -5.49284  ,
        -11.728105 ,  -11.742494 ,   -5.990808 ,   -2.6591134],
      dtype=float32)

In [ ]:
data_dir = pathlib.Path('samples')
filenames = list(data_dir.glob('**/*.wav'))
filenames = [str(f) for f in filenames]
print(filenames)
y = np.array([f[8:11] for f in filenames])
y

['samples/e0.wav', 'samples/dm2.wav', 'samples/c1.wav', 'samples/e4.wav', 'samples/am2.wav', 'samples/g3.wav', 'samples/dm0.wav', 'samples/dm1.wav', 'samples/e1.wav', 'samples/am1.wav', 'samples/dm3.wav', 'samples/c0.wav', 'samples/g2.wav', 'samples/c3.wav', 'samples/e2.wav', 'samples/g4.wav', 'samples/g1.wav', 'samples/c2.wav', 'samples/e3.wav', 'samples/am4.wav', 'samples/g0.wav', 'samples/am0.wav', 'samples/am3.wav', 'samples/dm4.wav', 'samples/c4.wav']


array(['e0.', 'dm2', 'c1.', 'e4.', 'am2', 'g3.', 'dm0', 'dm1', 'e1.',
       'am1', 'dm3', 'c0.', 'g2.', 'c3.', 'e2.', 'g4.', 'g1.', 'c2.',
       'e3.', 'am4', 'g0.', 'am0', 'am3', 'dm4', 'c4.'], dtype='<U3')

In [ ]:
print(data_dir)

samples


In [ ]:
y

array(['e0.', 'dm2', 'c1.', 'e4.', 'am2', 'g3.', 'dm0', 'dm1', 'e1.',
       'am1', 'dm3', 'c0.', 'g2.', 'c3.', 'e2.', 'g4.', 'g1.', 'c2.',
       'e3.', 'am4', 'g0.', 'am0', 'am3', 'dm4', 'c4.'], dtype='<U3')

In [ ]:
df = pd.DataFrame(columns=list("0123456789ABCDEFGHIJ"))

for i in filenames:
    signal, sr = librosa.load('/content/' + i)
    C = librosa.feature.mfcc(y=signal, sr=sr)
    coefs = np.average(C, axis = 1)
    # fft = list(abs(scipy.fft(signal)))
    # fft.sort()
    df.loc[len(df)] = coefs




from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


scaled_features = scaler.fit_transform(df)

df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
print(df)
df['chord'] = [re.sub(r'[0-9]', '', w).replace('.', '') for w in y]

           0         1         2  ...         H         I         J
0   0.599704  0.649918  0.452457  ...  0.604872  0.230275  0.262550
1   0.535568  0.595734  1.000000  ...  0.513632  0.560369  0.590034
2   0.169309  0.564090  0.704905  ...  0.183106  0.000000  0.443040
3   0.595643  0.467628  0.494282  ...  0.443877  0.547678  0.297883
4   0.485546  0.201145  0.444888  ...  0.114521  0.566028  0.296030
5   0.000000  0.247189  0.778170  ...  0.705193  0.943549  0.098743
6   0.619421  0.340780  0.421694  ...  0.454362  0.910369  1.000000
7   0.892653  0.392818  0.442825  ...  0.513344  0.259632  0.623840
8   0.748624  0.149563  0.789215  ...  0.494329  0.781695  0.624350
9   0.464435  0.541552  0.741330  ...  0.000000  0.680691  0.556229
10  0.435966  0.485811  0.827973  ...  0.470628  0.565610  0.588900
11  0.193110  0.571113  0.701064  ...  0.174322  0.187798  0.453837
12  0.223358  0.414210  0.460030  ...  0.726009  0.469351  0.000000
13  0.285373  0.800576  0.497240  ...  0.084064 

In [ ]:
train_mean = df.mean().values

In [ ]:
train_mean

array([0.49444085, 0.46847287, 0.56879884, 0.53753573, 0.45828167,
       0.5357371 , 0.55867004, 0.50949186, 0.4309364 , 0.44809261,
       0.48810646, 0.45533407, 0.31903142, 0.49106371, 0.46958759,
       0.60542673, 0.4677121 , 0.41692969, 0.51788574, 0.41125706])

In [ ]:
train = df.drop([4, 9, 14, 19, 24],axis='index')
test =   df[df.index.isin([4, 9, 14, 19, 24])]

In [ ]:
X_train = df.drop(columns = 'chord')
y_train = df['chord']


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
GOD = KNeighborsClassifier(n_neighbors=3)
GOD.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
# !unzip '/content/drive/MyDrive/Chords/dataset-easy.zip' 

In [ ]:
# test_dir = pathlib.Path('dataset-easy')
# testnames = list(test_dir.glob('**/*.wav'))
# testnames = [str(f) for f in testnames]
# testnames = sorted(testnames, key=lambda x: int(x[-8:-4]))

# for i in testnames[:921]:
#     signal, sr = librosa.load('/content/' + i)
#     C = librosa.feature.mfcc(y=signal, sr=sr)
#     coefs = np.average(C, axis = 1)
#     # fft = list(abs(scipy.fft(signal)))
#     # fft.sort()
#     scaled_coefs = scaler.transform(coefs.reshape(1, -1))
    
#     X_train.loc[len(X_train)] = scaled_coefs.tolist()[0]
#     y_train.loc[len(y_train)]= GOD.predict(scaled_coefs)[0]
    
#     print(i[-8: -4], GOD.predict(scaled_coefs)[0])


In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F,G,H,I,J
0,0.599704,0.649918,0.452457,0.249618,0.129190,0.335486,0.067291,0.000000,0.408713,0.951795,0.944660,0.597390,0.065887,0.478850,0.380000,0.695834,0.611888,0.604872,0.230275,0.262550
1,0.535568,0.595734,1.000000,0.558741,0.399755,0.699514,0.656262,0.435193,0.292404,0.395427,0.525929,0.456095,0.111508,0.108720,0.146384,0.000000,0.297559,0.513632,0.560369,0.590034
2,0.169309,0.564090,0.704905,0.755912,0.414622,0.313728,0.498184,0.685533,0.201312,0.381958,0.479089,0.227769,0.747424,0.698039,0.323397,0.318206,0.284431,0.183106,0.000000,0.443040
3,0.595643,0.467628,0.494282,0.382141,0.158881,0.270289,0.132143,0.044898,0.240413,0.692621,0.955420,0.692293,0.036008,0.349485,0.608001,0.836988,0.506258,0.443877,0.547678,0.297883
4,0.485546,0.201145,0.444888,1.000000,0.673653,0.485832,0.821418,0.929673,0.672240,0.030527,0.241140,0.446926,0.234550,0.498238,0.496059,0.569858,0.178788,0.114521,0.566028,0.296030
5,0.000000,0.247189,0.778170,0.653132,0.638372,0.917184,1.000000,0.657234,0.281716,0.384406,0.395247,0.566463,0.195491,0.573931,0.750584,0.936968,0.776437,0.705193,0.943549,0.098743
6,0.619421,0.340780,0.421694,0.249081,0.341989,0.310996,0.226229,0.499159,0.728933,0.407141,0.462094,0.454972,0.000000,0.000000,0.000000,0.077146,0.320822,0.454362,0.910369,1.000000
7,0.892653,0.392818,0.442825,0.578089,0.729912,1.000000,0.687662,0.955114,0.592373,0.598823,0.023304,0.270833,0.400683,0.161711,0.206510,0.214330,0.492932,0.513344,0.259632,0.623840
8,0.748624,0.149563,0.789215,0.573246,0.070638,0.000000,0.158583,0.073361,0.033998,0.682278,1.000000,1.000000,0.462503,0.340078,0.593120,1.000000,0.588603,0.494329,0.781695,0.624350
9,0.464435,0.541552,0.741330,0.632571,0.669640,0.730119,0.737296,0.703067,0.667166,0.341795,0.000000,0.000000,0.277238,0.633751,0.643653,0.365642,0.000000,0.000000,0.680691,0.556229


In [ ]:
y_train

0      e
1     dm
2      c
3      e
4     am
5      g
6     dm
7     dm
8      e
9     am
10    dm
11     c
12     g
13     c
14     e
15     g
16     g
17     c
18     e
19    am
20     g
21    am
22    am
23    dm
24     c
Name: chord, dtype: object

In [ ]:
# GOD = KNeighborsClassifier(n_neighbors=3)
# GOD.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
!unzip '/content/drive/MyDrive/Chords/dataset-hard.zip' 

In [ ]:
from librosa.core import istft


test_dir = pathlib.Path('dataset-hard')
testnames = list(test_dir.glob('**/*.wav'))
testnames = [str(f) for f in testnames]
testnames = sorted(testnames, key=lambda x: int(x[-8:-4]))

test_mean = np.zeros(train_mean.shape)
for i in testnames[:1000]:
    signal, sr = librosa.load('/content/' + i)
    
    C = librosa.feature.mfcc(y=signal, sr=sr)
    coefs = np.average(C, axis = 1)
    scaled_coefs = scaler.transform(coefs.reshape(1, -1))
    test_mean = test_mean + scaled_coefs / 1000

In [ ]:
test_mean 


array([[ 1.40602004,  0.16231493, -0.02223775,  0.85247494,  0.0569685 ,
        -0.04192612,  0.10659984,  0.25262091,  0.35855151,  0.46567939,
         0.61297562,  0.68129814, -0.16938381,  0.14044128,  0.20773975,
         0.38641745,  0.57405188,  0.68220731,  0.76075373,  0.34768621]])

In [ ]:
train_mean

array([0.49444085, 0.46847287, 0.56879884, 0.53753573, 0.45828167,
       0.5357371 , 0.55867004, 0.50949186, 0.4309364 , 0.44809261,
       0.48810646, 0.45533407, 0.31903142, 0.49106371, 0.46958759,
       0.60542673, 0.4677121 , 0.41692969, 0.51788574, 0.41125706])

In [ ]:
noise = test_mean - train_mean

In [ ]:
noise

array([[ 0.91157919, -0.30615794, -0.59103659,  0.31493921, -0.40131317,
        -0.57766322, -0.4520702 , -0.25687095, -0.07238489,  0.01758678,
         0.12486916,  0.22596408, -0.48841523, -0.35062244, -0.26184784,
        -0.21900928,  0.10633978,  0.26527761,  0.24286798, -0.06357085]])

In [ ]:
from librosa.core import istft


test_dir = pathlib.Path('dataset-hard')
testnames = list(test_dir.glob('**/*.wav'))
testnames = [str(f) for f in testnames]
testnames = sorted(testnames, key=lambda x: int(x[-8:-4]))

for i in testnames[4694:]:
    signal, sr = librosa.load('/content/' + i)

    C = librosa.feature.mfcc(y=signal, sr=sr)
    coefs = np.average(C, axis = 1)
    # fft = list(abs(scipy.fft(signal)))
    # fft.sort()
    scaled_coefs = scaler.transform(coefs.reshape(1, -1))
    scaled_coefs = scaled_coefs - noise  
    print(i[-8: -4], GOD.predict(scaled_coefs)[0])

4694 e
4695 g
4696 c
4697 c
4698 e
4699 dm
4700 c
4701 e
4702 e
4703 e
4704 e
4705 am
4706 dm
4707 c
4708 c
4709 g
4710 dm
4711 e
4712 am
4713 am
4714 am
4715 g
4716 c
4717 dm
4718 dm
4719 dm
4720 c
4721 g
4722 c
4723 c
4724 e
4725 dm
4726 dm
4727 c
4728 e
4729 am
4730 g
4731 am
4732 am
4733 g
4734 am
4735 e
4736 e
4737 am
4738 e
4739 c
4740 e
4741 dm
4742 c
4743 e
4744 g
4745 am
4746 am
4747 dm
4748 e
4749 c
4750 am
4751 g
4752 am
4753 c
4754 c
4755 dm
4756 g
4757 g
4758 g
4759 e
4760 g
4761 dm
4762 dm
4763 g
4764 e
4765 dm
4766 c
4767 c
4768 g
4769 g
4770 c
4771 dm
4772 g
4773 c
4774 c
4775 g
4776 am
4777 am
4778 g
4779 e
4780 e
4781 g
4782 am
4783 g
4784 g
4785 g
4786 dm
4787 g
4788 g
4789 am
4790 c
4791 dm
4792 c
4793 g
4794 dm
4795 g
4796 e
4797 c
4798 c
4799 c
4800 g
4801 g
4802 dm
4803 c
4804 c
4805 g
4806 e
4807 c
4808 c
4809 g
4810 e
4811 g
4812 dm
4813 g
4814 c
4815 dm
4816 c
4817 c
4818 dm
4819 g
4820 c
4821 dm
4822 c
4823 dm
4824 dm
4825 g
4826 am
4827 dm
4828 g
4829 am
483

In [ ]:
/content/dataset-easy/0000.wav

In [ ]:
for i in testnames:
    s_rate, signal = wavfile.read('/content/' + i)
    fft = list(abs(scipy.fft(signal)))
    fft.sort()
    df.loc[len(df)] = fft[-10:]
